In [1]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, BoxAnnotation, Span
from bokeh.io import output_notebook, reset_output
from bokeh.layouts import grid

# Import from statsmodels
# Documentation: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.STL.html#statsmodels.tsa.seasonal.STL
from statsmodels.tsa.seasonal import STL


In [2]:
# Initialize an empty list to store sample data
sample_data = []

# Open the sample file in read mode
with open("sample_2465_2524.txt", "r") as sample_file:
    # Loop through each line in the file
    for line in sample_file:
        # Convert the line to a float and append it to the sample_data list
        sample_data.append(float(line))


In [3]:
# Create an instance of the STL class and fit it to the sample_data
# period is the length of the seasonal component
# robust parameter indicates whether to use robust fitting (True) or not (False)
# inner_iter(5) and outer_iter(15) are the numbers of iterations for the robust fitting
STL_data = STL(sample_data, period=10, robust=True).fit()


In [4]:
# Reset any previously set output settings
reset_output()

# Output the Bokeh plots directly in the notebook
output_notebook()

# Define tooltips for the hover tool
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "$x, $y"),
]

# Define the starting and ending points for the transient pattern
starting_point_tran = 2465
ending_point_tran = 2524

# Create a BoxAnnotation to highlight the transient pattern area
answer_box_annotation = BoxAnnotation(left=starting_point_tran, right=ending_point_tran, fill_alpha=0.2, fill_color="#FF5733")

# Create vertical spans to mark the starting and ending points of the transient pattern
starting_span = Span(location=starting_point_tran, dimension='height', line_color="#C70039", line_dash='4 4', line_width=2)
ending_span = Span(location=ending_point_tran, dimension='height', line_color="#C70039", line_dash='4 4', line_width=2)

# Create x-axis data points
x_axis_data = [*range(len(sample_data))]


Loading BokehJS ...

In [5]:
# Plotting raw data
# Create a figure for raw data
raw_plot = figure(tooltips=TOOLTIPS,  # Add tooltips for hover tool
                  title="Raw data", x_axis_label="time", y_axis_label="Flux")  # Set title and axis labels

# Create a ColumnDataSource for raw data
raw_datasource = ColumnDataSource(data=dict(x=x_axis_data, y=sample_data))

# Plot a line for raw data
raw_plot.line('x', 'y', source=raw_datasource, line_alpha=0.5, color="black", line_width=2, legend_label="Raw")

# Plot circles for raw data points
raw_plot.circle('x', 'y', source=raw_datasource, color="black", fill_alpha=0.6, legend_label="Raw", size=2)

# Add the BoxAnnotation and vertical spans to the raw plot
raw_plot.add_layout(answer_box_annotation)  # Add the box annotation
raw_plot.add_layout(starting_span)  # Add the starting vertical span
raw_plot.add_layout(ending_span)  # Add the ending vertical span

# Customize the appearance of the legend
raw_plot.legend.border_line_width = 3
raw_plot.legend.click_policy = "hide"
raw_plot.legend.border_line_color = "navy"
raw_plot.legend.border_line_alpha = 0.0


In [6]:
# Trend plot
trend_plot = figure(tooltips=TOOLTIPS,  # Add tooltips for hover tool
                    title="Trend Component", x_axis_label="Time", y_axis_label="Flux")  # Set title and axis labels

# Plot trend component
trend_plot.line(x_axis_data, STL_data.trend, line_alpha=0.5, color="black", line_width=2, legend_label="Trend")
trend_plot.circle(x_axis_data, STL_data.trend, color="black", fill_alpha=0.6, legend_label="Trend", size=2)

# Add the BoxAnnotation and vertical spans to the trend plot
trend_plot.add_layout(answer_box_annotation)  # Add the box annotation
trend_plot.add_layout(starting_span)  # Add the starting vertical span
trend_plot.add_layout(ending_span)  # Add the ending vertical span

# Sync x-axis range with the raw data plot
trend_plot.x_range = raw_plot.x_range


In [7]:
# Seasonal plot
seasonal_plot = figure(tooltips=TOOLTIPS,  # Add tooltips for hover tool
                       title="Seasonal Component", x_axis_label="Time", y_axis_label="Flux")  # Set title and axis labels

# Plot seasonal component
seasonal_plot.line(x_axis_data, STL_data.seasonal, line_alpha=0.5, color="black", line_width=2, legend_label="Seasonal")
seasonal_plot.circle(x_axis_data, STL_data.seasonal, color="black", fill_alpha=0.6, legend_label="Seasonal", size=2)

# Add the BoxAnnotation and vertical spans to the seasonal plot
seasonal_plot.add_layout(answer_box_annotation)  # Add the box annotation
seasonal_plot.add_layout(starting_span)  # Add the starting vertical span
seasonal_plot.add_layout(ending_span)  # Add the ending vertical span

# Sync x-axis range with the raw data plot
seasonal_plot.x_range = raw_plot.x_range


In [8]:
# Residual plot
residual_plot = figure(tooltips=TOOLTIPS,  # Add tooltips for hover tool
                       title="Residual Component", x_axis_label="Time", y_axis_label="Flux", sizing_mode='stretch_both')  # Set title and axis labels

# Plot residual component
residual_plot.line(x_axis_data, STL_data.resid, line_alpha=0.5, color="black", line_width=2, legend_label="Residual")
residual_plot.circle(x_axis_data, STL_data.resid, color="black", fill_alpha=0.6, legend_label="Residual", size=2)

# Add the BoxAnnotation and vertical spans to the residual plot
residual_plot.add_layout(answer_box_annotation)  # Add the box annotation
residual_plot.add_layout(starting_span)  # Add the starting vertical span
residual_plot.add_layout(ending_span)  # Add the ending vertical span

# Sync x-axis range with the raw data plot
residual_plot.x_range = raw_plot.x_range


In [9]:
# Create a grid layout for the plots
grid_layout = grid([[raw_plot, trend_plot], [seasonal_plot, residual_plot]])

# Display the combined grid plot
show(grid_layout)